In [2]:
# Add the parent directory to Python path so we can import from core
import sys
import os
# Get the current working directory and go up one level to the backend directory
backend_dir = os.path.dirname(os.getcwd())
if backend_dir not in sys.path:
    sys.path.append(backend_dir)

In [3]:
import asyncio, datetime, uuid
from core.issue import IssueContext
from core.llm import LLMClient

llm_client = LLMClient()
issue = IssueContext(llm_client)

issue.start()

# Let background tasks spin up without blocking the event loop
await asyncio.sleep(1)

# send a message to the issue
issue.ingest({
    "id": str(uuid.uuid4()),
    "type": "issue.begin",
    "payload": {
        "id": "test_1",
        "name": "Issue Description",
        "description": "This is the issue description initially given by the user.",
        "rationale": "",
        "outcomes": {},
        "result": "My truck revs up when I put the keys in the ignition but it won't start."
    },
    "issue_id": issue.id,
    "meta": {
        "ts": datetime.datetime.utcnow().isoformat(),
        "source": "system"
    }
})

# Keep the notebook cell alive briefly to stream prints from tasks
await asyncio.sleep(600)

await issue.stop()



Ingesting message: id='f04b2014-eeae-4d45-b181-d633c7bdf523' type='issue.begin' v=1 issue_id='838751eb-f3a8-4625-8beb-38e4040bdd90' payload={'id': 'test_1', 'name': 'Issue Description', 'description': 'This is the issue description initially given by the user.', 'rationale': '', 'outcomes': {}, 'result': "My truck revs up when I put the keys in the ignition but it won't start."} meta={'ts': '2025-09-11T18:55:20.084229', 'source': 'system'} ts='2025-09-11T18:55:20.084229' source='test'
Executing handler for event type: issue.begin


C:\Users\jakej\AppData\Local\Temp\ipykernel_11820\3192048691.py:27: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "ts": datetime.datetime.utcnow().isoformat(),


We have no prior hypotheses. We need to create a list of possible issues that could cause: truck revs up (idle) when key in ignition but won't start. That's a common symptom of starting issues: battery, alternator, starter, fuel, spark, etc. We should assign initial probabilities. Use simple estimation. Let's create plausible hypotheses: "Bad Starter", "Weak Battery", "Fuel Pump Failure", "Ignition Coil Failure", "Bad Spark Plugs", "Engine Seized", "Low Engine Oil", "Faulty Ignition Switch". Provide short descriptions. Assign base probability scores: maybe use 1.0 as base for each? But we need unnormalized scores. Could give different initial scores. For example:

- Bad Starter: 1.5
- Weak Battery: 1.2
- Fuel Pump Failure: 1.0
- Ignition Coil Failure: 0.8
- Bad Spark Plugs: 0.6
- Engine Seized: 0.5
- Low Engine Oil: 0.4
- Faulty Ignition Switch: 0.3

We have only one test, the issue description. That test provides some support for many. The outcome "revs up when I put the keys in the i

CancelledError: 